In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
#sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
#sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
#sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis' )
sys.path.insert(1, 'C:\\Users\\SJLab_Slice\\Documents\\photometry_analysis\\freely_moving_photometry_analysis')

import os
import joypy
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.signal import decimate
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean, plot_one_side, calculate_error_bars
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.individual_trial_analysis_utils import ZScoredTraces, SessionData, CueAlignedData, CueAlignedSidedData, ChoiceAlignedData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
mouse = 'YJ_03'
date = '20220607'
saving_folder = "W:\\swc\\sjones\\users\\Yvonne\\photometry_2AC\\processed_data\\" + mouse + '\\'
restructured_data_filename = mouse + '_' + date + '_' + 'restructured_data.pkl'
trial_data = pd.read_pickle(saving_folder + restructured_data_filename) 

In [7]:
demod_signal = np.load(saving_folder + mouse + '_' + date + "_smoothed_signal.npy" )
demod_signal.shape
plt.plot(demod_signal)

In [72]:
trial_data

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,TrialCueDelay,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
0,0.0,1.0,1.0,1.0,1.0,TrialStart,23.9921,24.0021,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
1,0.0,1.0,2.0,1.0,2.0,WaitForPoke,24.0021,58.6612,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
2,0.0,1.0,3.0,1.0,2.0,CueDelay,58.6612,58.6614,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
3,0.0,1.0,2.0,2.0,2.0,WaitForPoke,58.6614,58.6615,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
4,0.0,1.0,3.0,2.0,2.0,CueDelay,58.6615,58.7615,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5934,415.0,7.0,3.0,3.0,3.0,CueDelay,3674.2693,3674.3693,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0
5935,415.0,7.0,4.0,1.0,1.0,WaitForPortOut,3674.3693,3674.3744,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0
5936,415.0,7.0,5.0,1.0,1.0,WaitForResponse,3674.3744,3674.8657,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0
5937,415.0,7.0,7.0,1.0,1.0,RightReward,3674.8657,3674.8800,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0


In [83]:
trial_data.values

array([[0.0, 1.0, 1.0, ..., 2.0, 0.0, 0.0],
       [0.0, 1.0, 2.0, ..., 2.0, 0.0, 0.0],
       [0.0, 1.0, 3.0, ..., 2.0, 0.0, 0.0],
       ...,
       [415.0, 7.0, 5.0, ..., 2.0, 0.0, 1.0],
       [415.0, 7.0, 7.0, ..., 2.0, 0.0, 1.0],
       [415.0, 7.0, 8.0, ..., 2.0, 0.0, 1.0]], dtype=object)

In [136]:
#print(trial_data[trial_data['State name']== 'LeftReward'].shape[0])
#print(trial_data[trial_data['State name']== 'RightReward'].shape)
#print(trial_data[trial_data['State name']== 'Drinking'].shape)


print(trial_data[trial_data['Trial num']== 0].shape)  # how many events in trial 1?
print(trial_data.groupby(['Trial num']).size())       # print all trials and for each one how many rows (=events)
print('   ')
print(trial_data.groupby(['State type', 'State name']).size())
print('   ')
print(trial_data[trial_data['Trial num']<2].size)  


#print(trial_data.groupby(['State type']).size())
#print(trial_data.groupby(['Trial num']).size())

#print(trial_data.groupby(['Trial num']).size())




(9, 18)
Trial num
0.0       9
1.0      23
2.0      14
3.0      11
4.0      10
         ..
411.0    11
412.0    10
413.0    10
414.0    17
415.0    11
Length: 416, dtype: int64
   
State type  State name            
1.0         TrialStart                 416
2.0         WaitForPoke               1801
3.0         CueDelay                  1801
4.0         WaitForPortOut             416
5.0         WaitForResponse            416
5.5         First incorrect choice      43
6.0         LeftReward                 183
7.0         RightReward                190
8.0         Drinking                   373
8.5         Leaving reward port        257
9.0         Punish                      43
dtype: int64
   
576


In [122]:
trial_data[trial_data['State type']==3]

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,TrialCueDelay,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
2,0.0,1.0,3.0,1.0,2.0,CueDelay,58.6612,58.6614,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
4,0.0,1.0,3.0,2.0,2.0,CueDelay,58.6615,58.7615,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
11,1.0,7.0,3.0,1.0,9.0,CueDelay,65.1534,65.1536,0.1,2.0,2.0,0.0,62.4978,147.2551,1.0,2.0,0.0,1.0
13,1.0,7.0,3.0,2.0,9.0,CueDelay,65.1537,65.1990,0.1,2.0,2.0,0.0,62.4978,147.2551,1.0,2.0,0.0,1.0
15,1.0,7.0,3.0,3.0,9.0,CueDelay,87.5159,87.5160,0.1,2.0,2.0,0.0,62.4978,147.2551,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5921,414.0,1.0,3.0,5.0,6.0,CueDelay,3657.0530,3657.0532,0.1,1.0,1.0,1.0,3653.0277,3658.0229,1.0,1.0,0.0,1.0
5923,414.0,1.0,3.0,6.0,6.0,CueDelay,3657.0533,3657.1533,0.1,1.0,1.0,1.0,3653.0277,3658.0229,1.0,1.0,0.0,1.0
5930,415.0,7.0,3.0,1.0,3.0,CueDelay,3674.2688,3674.2689,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0
5932,415.0,7.0,3.0,2.0,3.0,CueDelay,3674.2690,3674.2692,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0


In [80]:
trial_data[trial_data['State type']==3]

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,TrialCueDelay,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
2,0.0,1.0,3.0,1.0,2.0,CueDelay,58.6612,58.6614,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
4,0.0,1.0,3.0,2.0,2.0,CueDelay,58.6615,58.7615,0.1,2.0,-1.0,-1.0,23.9921,62.2970,0.0,2.0,0.0,0.0
11,1.0,7.0,3.0,1.0,9.0,CueDelay,65.1534,65.1536,0.1,2.0,2.0,0.0,62.4978,147.2551,1.0,2.0,0.0,1.0
13,1.0,7.0,3.0,2.0,9.0,CueDelay,65.1537,65.1990,0.1,2.0,2.0,0.0,62.4978,147.2551,1.0,2.0,0.0,1.0
15,1.0,7.0,3.0,3.0,9.0,CueDelay,87.5159,87.5160,0.1,2.0,2.0,0.0,62.4978,147.2551,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5921,414.0,1.0,3.0,5.0,6.0,CueDelay,3657.0530,3657.0532,0.1,1.0,1.0,1.0,3653.0277,3658.0229,1.0,1.0,0.0,1.0
5923,414.0,1.0,3.0,6.0,6.0,CueDelay,3657.0533,3657.1533,0.1,1.0,1.0,1.0,3653.0277,3658.0229,1.0,1.0,0.0,1.0
5930,415.0,7.0,3.0,1.0,3.0,CueDelay,3674.2688,3674.2689,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0
5932,415.0,7.0,3.0,2.0,3.0,CueDelay,3674.2690,3674.2692,0.1,2.0,1.0,1.0,3658.2139,3674.8801,1.0,2.0,0.0,1.0


In [22]:
trial_data[trial_data['State type']==3]['Time start'].values

array([  58.6612,   58.6615,   65.1534, ..., 3674.2688, 3674.269 ,
       3674.2693])

In [47]:
choices = trial_data[trial_data['State type'] == 5]['Time end'].values
final_cue = trial_data[(trial_data['State type'] == 3) & (trial_data['Instance in state'] == trial_data['Max times in state'])]['Time end'].values
all_cues = trial_data[(trial_data['State type'] == 3)]['Time end'].values
time = np.linspace(0,demod_signal.shape[0]/10000, demod_signal.shape[0])
plt.plot(time, demod_signal)
plt.scatter(final_cue, np.ones(final_cue.shape)*0.02, color='r', s=6)
plt.scatter(all_cues, np.ones(all_cues.shape)*0.025, color='g', s=6)

In [158]:
# YVONNE LEARNING, not useful

l_drinking = trial_data[(trial_data['State type']==8) & (trial_data['Response']==1)]['Time start'].values
r_drinking = trial_data[(trial_data['State type']==8) & (trial_data['Response']==2)]['Time start'].values

l_reward = trial_data[(trial_data['State type']==6)]['Time start'].values
r_reward = trial_data[(trial_data['State type']==7)]['Time start'].values

plt.plot(time, demod_signal)
plt.scatter(l_drinking, np.ones(l_drinking.shape)*0.02, color='g', s=6)# drinking left
plt.scatter(r_drinking, np.ones(r_drinking.shape)*0.02, color='r', s=6)# drinking right

plt.scatter(l_reward, np.ones(l_reward.shape)*0.023, color='g', s=6)# reward left
plt.scatter(r_reward, np.ones(r_reward.shape)*0.023, color='r', s=6)# reward right

print(l_drinking.size)
print(r_drinking.size)

# drinking left = drinking + response = 1

test = trial_data[(trial_data['Trial type']==1)]['Time start'].values
plt.scatter(test, np.ones(test.shape)*0.026, color='b', s=6)# reward right




183
190


In [159]:
drinking = trial_data[trial_data['State type']==8]['Time start'].values
print(drinking.size)
#print(drinking)



373


In [61]:
class CustomAlignedData(object):
    def __init__(self, session_data, params):
        saving_folder = "W:\\swc\\sjones\\users\\Yvonne\\photometry_2AC\\processed_data\\" + session_data.mouse + '\\'
        #saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + session_data.mouse + '\\'
        restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = session_data.mouse + '_' + session_data.date + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)

        fiber_options = np.array(['left', 'right'])
        fiber_side_numeric = (np.where(fiber_options == session_data.fiber_side)[0] + 1)[0]
        contra_fiber_side_numeric = (np.where(fiber_options != session_data.fiber_side)[0] + 1)[0]

        self.ipsi_data = ZScoredTraces(trial_data, dff, params, fiber_side_numeric, 0)
        #self.ipsi_data.get_peaks()
        self.contra_data = ZScoredTraces(trial_data, dff,params, contra_fiber_side_numeric, 0)
        #self.contra_data.get_peaks()

In [52]:
def get_all_experimental_records():
    experiment_record = pd.read_csv("W:\\swc\\sjones\\users\\Yvonne\\photometry_2AC\\experimental_record_yvonne.csv")
    experiment_record['date'] = experiment_record['date'].astype(str)
    return experiment_record

In [56]:
def open_experiment(experiment_to_add):
    for index, experiment in experiment_to_add.iterrows():
        saving_folder = "W:\\swc\\sjones\\users\\Yvonne\\photometry_2AC\\processed_data\\" + experiment['mouse_id'] + '\\'
        #saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + experiment['mouse_id']  + '\\'
        restructured_data_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)
        session_traces = SessionData(experiment['fiber_side'], experiment['recording_site'], experiment['mouse_id'], experiment['date'])
    return session_traces


In [57]:
mouse_id = 'YJ_03'
date = '20220607'
recording_site = 'striatum'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & 
                                        (all_experiments['mouse_id'] == mouse_id) &
                                        (all_experiments['recording_site'] == recording_site)]
session_data = open_experiment(experiment_to_process)

In [59]:
saving_folder = "W:\\swc\\sjones\\users\\Yvonne\\photometry_2AC\\processed_data\\" + mouse_id + '\\'
#saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + session_data.mouse + '\\'
restructured_data_filename = mouse_id + '_' + date + '_' + 'restructured_data.pkl'
trial_data = pd.read_pickle(saving_folder + restructured_data_filename)

In [62]:

params = {'state_type_of_interest': 5, # 5.5 = first incorrect choice
    'outcome': 1, # correct or incorrect: 0 = incorrect, 1 = correct, 2 = both
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1, # 0 = dont care, 1 = state only entered once,
    'last_response': 0, # trial before: 0 = dont care. 1 = left, 2 = right
    'align_to' : 'Time start', # time end or time start
    'instance': 0, # only for no repeats = 0, -1 = last instance, 1 = first instance
    'plot_range': [-1, 1],
    'first_choice_correct': 1, # useful for non-punished trials 0 = dont care, 1 = only correct trials, (-1 = incorrect trials)
    'cue': None}
processed_data = CustomAlignedData(session_data, params)

(5939, 18)
(5939, 18)


In [64]:
processed_data.contra_data.event_times

array([ 174.4789,  183.9935,  188.2172,  191.0954,  194.0431,  196.2772,
        208.0957,  211.2988,  216.1861,  218.2812,  241.5543,  265.2193,
        300.041 ,  310.4776,  321.787 ,  326.3255,  340.2828,  364.5968,
        368.2683,  372.0313,  379.1402,  381.418 ,  383.8203,  389.5358,
        397.3552,  406.6721,  410.6497,  423.4577,  428.8773,  439.4165,
        467.8739,  471.3615,  481.6611,  504.9134,  507.9613,  511.0065,
        527.0039,  539.6043,  551.2279,  558.4217,  560.8807,  611.6233,
        614.8413,  620.4255,  623.1099,  655.945 ,  670.3243,  675.8354,
        681.089 ,  698.9489,  706.7124,  714.6192,  729.285 ,  732.6877,
        746.3806,  748.9459,  755.6291,  766.552 ,  781.1351,  801.4488,
        814.6256,  817.2877,  822.5672,  846.9554,  850.1776,  852.7876,
        864.4563,  879.6079,  882.1006,  889.9522,  911.6287,  916.7433,
        935.0991,  937.0049,  940.4477,  943.4886,  945.4398,  960.9001,
        991.6344,  997.366 , 1000.5982, 1004.2112, 

In [67]:
processed_data.contra_data.sorted_traces.shape

(183, 160000)

In [70]:
processed_data.contra_data.trial_nums

array([  7.,   8.,   9.,  10.,  11.,  12.,  15.,  16.,  17.,  18.,  23.,
        27.,  28.,  31.,  33.,  34.,  38.,  41.,  42.,  43.,  45.,  46.,
        47.,  49.,  51.,  53.,  54.,  57.,  59.,  61.,  68.,  69.,  72.,
        78.,  79.,  80.,  84.,  87.,  90.,  92.,  93., 101., 102., 104.,
       105., 111., 113., 114., 116., 120., 122., 124., 128., 129., 132.,
       133., 135., 137., 140., 142., 143., 144., 145., 151., 152., 153.,
       156., 159., 160., 162., 166., 167., 169., 170., 171., 172., 173.,
       177., 183., 184., 185., 186., 187., 199., 209., 210., 214., 215.,
       216., 223., 225., 226., 229., 231., 232., 233., 234., 236., 238.,
       239., 241., 244., 246., 250., 252., 258., 260., 261., 262., 264.,
       271., 275., 281., 282., 285., 291., 293., 295., 296., 299., 300.,
       309., 310., 312., 313., 314., 315., 316., 317., 318., 321., 323.,
       324., 325., 326., 327., 330., 331., 332., 333., 334., 337., 338.,
       341., 342., 344., 346., 347., 348., 349., 35

In [69]:
plt.imshow(processed_data.contra_data.sorted_traces, aspect='auto')

In [38]:
heat_map_and_mean(processed_data, sort=True, error_bar_method='sem', mean_across_mice=False, cue_vline=0, xlims=[-2,3])
OutputDir = "W:\\users\\Yvonne\\photometry_2AC\\figures_FGscript\\" + mouse + "\\" 
#OutputDir = '/home/matthew/Documents/figures/SNL_photo/' + mouse_id

if not os.path.isdir(OutputDir):
    os.mkdir(OutputDir)

if params['state_type_of_interest'] == 3: 
    plt.savefig(OutputDir + '/' + mouse_id + '_' + date + '_' + recording_site + '_' + 'aligned_cue' + '.png') 
elif params['state_type_of_interest'] == 5:
    if params['align_to'] == 'Time end':
        plt.savefig(OutputDir + '/' + mouse_id + '_' + date + '_' + recording_site + '_' + 'aligned_reward' + '.png')
    elif params['align_to'] == 'Time start':
        plt.savefig(OutputDir + '/' + mouse_id + '_' + date + '_' + recording_site + '_' + 'aligned_movement' + '.png')

NameError: name 'get_axes' is not defined

In [24]:
test.ipsi_data.time_points

array([-8.    , -7.9999, -7.9998, ...,  7.9998,  7.9999,  8.    ])

In [12]:
trial_data.loc[60:70]

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,TrialCueDelay,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
60,2.0,1.0,8.5,1.0,1.0,Leaving reward port,37.9120,38.1112,NaN,1.0,2.0,1.0,NaN,NaN,1.0,1.0,0.0,1.0
61,3.0,1.0,1.0,1.0,1.0,TrialStart,38.1095,38.1195,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
62,3.0,1.0,2.0,1.0,9.0,WaitForPoke,38.1195,47.0148,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
63,3.0,1.0,3.0,1.0,9.0,CueDelay,47.0148,47.0149,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
64,3.0,1.0,2.0,2.0,9.0,WaitForPoke,47.0149,47.0153,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
65,3.0,1.0,3.0,2.0,9.0,CueDelay,47.0153,47.0154,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
66,3.0,1.0,2.0,3.0,9.0,WaitForPoke,47.0154,47.0156,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
67,3.0,1.0,3.0,3.0,9.0,CueDelay,47.0156,47.0157,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
68,3.0,1.0,2.0,4.0,9.0,WaitForPoke,47.0157,47.0158,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0
69,3.0,1.0,3.0,4.0,9.0,CueDelay,47.0158,47.0159,0.1,1.0,1.0,1.0,38.1095,68.2379,1.0,1.0,0.0,1.0


# Analysis looking at sidedness vs cue id for Nacc


In [2]:
mouse_id = 'SNL_photo30'
date = '20201209'

In [3]:
cue_test = SessionData('left', 'Nacc', mouse_id, date)

In [4]:
cue_test_ev = CueAlignedSidedData(cue_test)

(586, 17)
(245, 160000)
(439, 17)
(246, 160000)
(586, 17)
(4, 160000)
(439, 17)
(239, 160000)
(586, 17)
(241, 160000)
(439, 17)
(7, 160000)


In [5]:
from utils.plotting import plot_one_side

In [6]:
ipsi_low = cue_test_ev.low_cue_ipsi_data
contra_low = cue_test_ev.low_cue_contra_data
ipsi_high = cue_test_ev.high_cue_ipsi_data
contra_high = cue_test_ev.high_cue_contra_data

In [357]:
light_pink = '#FFADCD'
light_blue = '#9BCFFD'
dark_pink = '#FF478B'
dark_blue = '#1F93FF'
fig1, ax1 = plt.subplots(1,1)
time_points = decimate(ipsi_low.time_points, 10)
mean_trace = decimate(ipsi_low.mean_trace, 10)
traces = decimate(ipsi_low.sorted_traces, 10)
ax1.plot(time_points, mean_trace, lw=1.5, label='ipsi low', color=light_blue)

mean_trace = decimate(contra_low.mean_trace, 10)
traces = decimate(contra_low.sorted_traces, 10)
ax1.plot(time_points, mean_trace, lw=1.5, label='contra low', color=dark_blue)

mean_trace = decimate(ipsi_high.mean_trace, 10)
traces = decimate(ipsi_high.sorted_traces, 10)
ax1.plot(time_points, mean_trace, lw=1.5, label='ipsi high', color=dark_pink)

mean_trace = decimate(contra_high.mean_trace, 10)
traces = decimate(contra_high.sorted_traces, 10)
ax1.plot(time_points, mean_trace, lw=1.5, label='contra high', color=light_pink)
plt.legend()



In [351]:
fig, axs = plt.subplots(nrows=4, ncols=2)
plot_one_side(ipsi_low, fig, axs[0,0], axs[0, 1], error_bar_method=None, sort=True, white_dot='reward')
plot_one_side(contra_high, fig, axs[1,0], axs[1, 1], error_bar_method=None, sort=True, white_dot='reward')
plot_one_side(ipsi_high, fig, axs[2,0], axs[2, 1], error_bar_method=None, sort=True, white_dot='reward')
plot_one_side(contra_low, fig, axs[3,0], axs[3, 1], error_bar_method=None, sort=True, white_dot='reward')


In [7]:
import seaborn as sns

In [8]:
a = decimate(ipsi_low.sorted_traces, 10)

In [9]:
time_points = decimate(ipsi_low.time_points, 10)

In [ ]:
sns.heatmap(a, xticklabels=time_points)

In [14]:
a.shape

(241, 16000)

In [15]:
time_points = decimate(ipsi_low.time_points, 10)